## Importar librerias y definición de la ruta  de trabajo (path)

In [162]:

import pandas as pd
import numpy as np
import os
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

%matplotlib inline

# Tratamiento de datos
# ==============================================================================
import shap
from skforecast.utils import load_forecaster
from skforecast.utils import save_forecaster
from skforecast.preprocessing import RollingFeatures
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import TimeSeriesFold
from skforecast.direct import ForecasterDirect
from skforecast.recursive import ForecasterRecursive
import skforecast
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import sklearn
from skforecast.datasets import fetch_dataset

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

# Modelado y Forecasting
# ==============================================================================


# Configuración warnings
# ==============================================================================
warnings.filterwarnings('once')

color = '\033[1m\033[38;5;208m'
print(f"{color}Versión skforecast: {skforecast.__version__}")
print(f"{color}Versión scikit-learn: {sklearn.__version__}")
print(f"{color}Versión pandas: {pd.__version__}")
print(f"{color}Versión numpy: {np.__version__}")

# Formato de los prints
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def headr(text):
    return ('\n'+color.UNDERLINE + text + color.END+'\n')


Versión skforecast: 0.15.1
Versión scikit-learn: 1.6.1
Versión pandas: 2.2.3
Versión numpy: 1.26.4


In [4]:
# Ingresa la ruta donde está el repositorio
ruta = 'c:/repo_remoto/'

## Preprocesado

In [106]:
mantenimiento_df = pd.read_csv(ruta + 'Etapa4/output/merge_df.csv')

In [202]:
equipo = 1  # Introduce el equipo a procesar
mantenimiento_df_1 = mantenimiento_df[mantenimiento_df['ID_Equipo'] == equipo].copy()
mantenimiento_df_1

,Fecha,ID_Equipo,Tipo_Mantenimiento,Ubicacion,Temperatura_C,Vibracion_mm_s,Horas_Operativas,Tipo_Equipo,Fabricante,Modelo,Potencia_kW,Horas_Recomendadas_Revision
0,2021-01-01,1,Preventivo,Ubicacion_C,34.749896,2.136812,614,Bomba,Fabricante_C,Modelo_6,173,913
500,2021-01-02,1,Preventivo,Ubicacion_C,89.588937,9.409404,634,Bomba,Fabricante_C,Modelo_6,173,913
1000,2021-01-03,1,Preventivo,Ubicacion_C,56.200558,1.529395,658,Bomba,Fabricante_C,Modelo_6,173,913
1500,2021-01-04,1,Preventivo,Ubicacion_C,70.555608,3.411009,664,Bomba,Fabricante_C,Modelo_6,173,913
2000,2021-01-05,1,Preventivo,Ubicacion_C,97.509000,0.618757,683,Bomba,Fabricante_C,Modelo_6,173,913
...,...,...,...,...,...,...,...,...,...,...,...,...
729247,2024-12-27,1,Preventivo,Ubicacion_C,38.587761,2.748949,808,Bomba,Fabricante_C,Modelo_6,173,913
729747,2024-12-28,1,Preventivo,Ubicacion_C,90.397197,5.382844,812,Bomba,Fabricante_C,Modelo_6,173,913
730247,2024-12-29,1,Preventivo,Ubicacion_C,21.104524,5.686564,824,Bomba,Fabricante_C,Modelo_6,173,913
730747,2024-12-30,1,Preventivo,Ubicacion_C,30.559628,7.948501,828,Bomba,Fabricante_C,Modelo_6,173,913


In [203]:
mantenimiento_df_1['Fecha'] = pd.to_datetime(mantenimiento_df_1['Fecha'], format='%Y-%m-%d')
mantenimiento_df_1 = mantenimiento_df_1.set_index('Fecha')

In [204]:
# Eliminar duplicados en el índice para evitar errores
mantenimiento_df_1 = mantenimiento_df_1[~mantenimiento_df_1.index.duplicated(keep='first')]

mantenimiento_df_1 = mantenimiento_df_1.asfreq('D')  # Convierte Timeseries a frecuencia especificada.
mantenimiento_df_1 = mantenimiento_df_1.sort_index()

In [205]:
# Sumatorio de filas duplicadas en mantenimiento_df
print("Sumatorio de filas duplicadas en mantenimiento_df:", mantenimiento_df_1.duplicated().sum())

Sumatorio de filas duplicadas en mantenimiento_df: 0


In [206]:
print(f'Número de filas con missing values: {mantenimiento_df_1.isnull().any(axis=1).mean()}')

Número de filas con missing values: 0.0


In [207]:
# Verificar que un índice temporal está completo

fecha_inicio = mantenimiento_df_1.index.min()
fecha_fin = mantenimiento_df_1.index.max()
date_range_completo = pd.date_range(start=fecha_inicio, end=fecha_fin, freq=mantenimiento_df_1.index.freq)

print(f"Índice completo: {(mantenimiento_df_1.index == date_range_completo).all()}")

Índice completo: True


In [208]:
mantenimiento_df_1

,ID_Equipo,Tipo_Mantenimiento,Ubicacion,Temperatura_C,Vibracion_mm_s,Horas_Operativas,Tipo_Equipo,Fabricante,Modelo,Potencia_kW,Horas_Recomendadas_Revision
Fecha,,,,,,,,,,,
2021-01-01,1,Preventivo,Ubicacion_C,34.749896,2.136812,614,Bomba,Fabricante_C,Modelo_6,173,913
2021-01-02,1,Preventivo,Ubicacion_C,89.588937,9.409404,634,Bomba,Fabricante_C,Modelo_6,173,913
2021-01-03,1,Preventivo,Ubicacion_C,56.200558,1.529395,658,Bomba,Fabricante_C,Modelo_6,173,913
2021-01-04,1,Preventivo,Ubicacion_C,70.555608,3.411009,664,Bomba,Fabricante_C,Modelo_6,173,913
2021-01-05,1,Preventivo,Ubicacion_C,97.509000,0.618757,683,Bomba,Fabricante_C,Modelo_6,173,913
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,1,Preventivo,Ubicacion_C,38.587761,2.748949,808,Bomba,Fabricante_C,Modelo_6,173,913
2024-12-28,1,Preventivo,Ubicacion_C,90.397197,5.382844,812,Bomba,Fabricante_C,Modelo_6,173,913
2024-12-29,1,Preventivo,Ubicacion_C,21.104524,5.686564,824,Bomba,Fabricante_C,Modelo_6,173,913


In [209]:
# Elimino ID_Orden, establezco Fecha como índice y le doy formato datetime
mantenimiento_df_1 = mantenimiento_df_1.drop(
    columns=['ID_Equipo','Ubicacion', 'Tipo_Equipo', 'Fabricante', 'Modelo', 'Potencia_kW', 'Horas_Recomendadas_Revision'])

In [210]:
mantenimiento_df_1

,Tipo_Mantenimiento,Temperatura_C,Vibracion_mm_s,Horas_Operativas
Fecha,,,,
2021-01-01,Preventivo,34.749896,2.136812,614
2021-01-02,Preventivo,89.588937,9.409404,634
2021-01-03,Preventivo,56.200558,1.529395,658
2021-01-04,Preventivo,70.555608,3.411009,664
2021-01-05,Preventivo,97.509000,0.618757,683
...,...,...,...,...
2024-12-27,Preventivo,38.587761,2.748949,808
2024-12-28,Preventivo,90.397197,5.382844,812
2024-12-29,Preventivo,21.104524,5.686564,824


In [211]:
from sklearn.preprocessing import LabelEncoder

# Crear un codificador para la columna Tipo_Mantenimiento
label_encoder = LabelEncoder()

# Transformar la columna Tipo_Mantenimiento en valores numéricos
mantenimiento_df_1['Tipo_Mantenimiento'] = label_encoder.fit_transform(mantenimiento_df_1['Tipo_Mantenimiento'])

# Verificar la transformación
mantenimiento_df_1

,Tipo_Mantenimiento,Temperatura_C,Vibracion_mm_s,Horas_Operativas
Fecha,,,,
2021-01-01,1,34.749896,2.136812,614
2021-01-02,1,89.588937,9.409404,634
2021-01-03,1,56.200558,1.529395,658
2021-01-04,1,70.555608,3.411009,664
2021-01-05,1,97.509000,0.618757,683
...,...,...,...,...
2024-12-27,1,38.587761,2.748949,808
2024-12-28,1,90.397197,5.382844,812
2024-12-29,1,21.104524,5.686564,824


In [331]:
# Separación datos train-test

steps = 400
datos_train = mantenimiento_df_1[:-steps]
datos_test  = mantenimiento_df_1[-steps:]
print(f"Fechas train : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
print(f"Fechas test  : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")

Fechas train : 2021-01-01 00:00:00 --- 2023-11-27 00:00:00  (n=1061)
Fechas test  : 2023-11-28 00:00:00 --- 2024-12-31 00:00:00  (n=400)


In [341]:
data = mantenimiento_df_1['Horas_Operativas']
target = mantenimiento_df_1['Tipo_Mantenimiento']

In [343]:
# Crear el forecaster con un modelo de regresión
forecaster = ForecasterRecursive(
    regressor=RandomForestRegressor(n_estimators=100, random_state=123),
    lags=470,
)

exogs_train = datos_train.drop(columns=['Horas_Operativas'])

# Entrenar el forecaster
forecaster.fit(y=data)
forecaster

# Hacer predicciones

exogs_test = datos_test.drop(columns=['Horas_Operativas'])

predicciones = forecaster.predict(steps=len(target))

print(predicciones.head(15), predicciones.shape)


2025-01-01    32.96
2025-01-02    42.52
2025-01-03    49.71
2025-01-04    50.54
2025-01-05    49.81
2025-01-06    49.86
2025-01-07    50.63
2025-01-08    49.93
2025-01-09    49.09
2025-01-10    48.66
2025-01-11    40.13
2025-01-12    44.46
2025-01-13    47.35
2025-01-14    48.31
2025-01-15    47.04
Freq: D, Name: pred, dtype: float64 (1461,)


In [344]:
# residuos
residuals = data.values - predicciones.values

print(residuals[0:], residuals.shape)

[ 581.04  591.48  608.29 ...  562.27  566.27 -239.73] (1461,)


In [345]:
def create_features_w_residuals(data, residuals, lags):
    features = pd.DataFrame(index=data.index)
    for lag in range(1, lags + 1):
        features[f'lag_{lag}'] = data.shift(lag)
    features['residuals'] = residuals
    features.dropna(inplace=True)

    return features

In [346]:
# Paso 3:
X = create_features_w_residuals(data, residuals, lags=470)
y = target.iloc[len(target) - len(X):]

print('X:\n', tabulate(X[:5], headers='keys', tablefmt='psql'), X.shape)

print('\ny:\n', y[:5], y.shape)

X:
 +---------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---

C:\Users\UF188430\AppData\Local\Temp\ipykernel_26632\4255506767.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[f'lag_{lag}'] = data.shift(lag)
C:\Users\UF188430\AppData\Local\Temp\ipykernel_26632\4255506767.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[f'lag_{lag}'] = data.shift(lag)
C:\Users\UF188430\AppData\Local\Temp\ipykernel_26632\4255506767.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [347]:
# Paso 4: Dividir en entrenamiento y prueba
steps = 15


X_train, X_test = X[:-steps], X[-steps:]
y_train, y_test = y[:-steps], y[-steps:]

In [348]:
# Paso 5: Ajustar bosque aleatorio en residuos
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [349]:
# Paso 6: generar predicciones

# Predecir los residuos con bosque aleatorio
y_pred = rf_model.predict(X_test)

print(y_pred)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]


In [350]:
# Paso 7: evaluar el modelo
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc}')

Accuracy: 1.0
